In [1]:
import pandas as pd
import numpy as np
from bs4 import BeautifulSoup

from selenium import webdriver
import warnings
warnings.filterwarnings('ignore')
import time 
from tqdm import tqdm

In [3]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('C:/Users/YONSAI/Downloads/chromedriver', options=options)

songIds = []
for i in tqdm(range(1964, 2023)):
    url = f'https://www.melon.com/chart/age/index.htm?chartType=YE&chartGenre=KPOP&chartDate={i}' 
    driver.get(url)
    time.sleep(2)

    soup = BeautifulSoup(driver.page_source,'html.parser')
    trs = soup.select('tbody > tr.lst50')

    for tr in trs:
        if tr.select_one('.bullet_icons.age_19'):
            continue
        songId  = tr.select_one('.btn.btn_icon_detail')['onclick'].split("'")[-2]
        songIds.append(songId)
songIds[:5],len(songIds)

100%|██████████| 59/59 [03:06<00:00,  3.17s/it]


(['4083218', '1003023', '1754987', '1002782', '1003568'], 2646)

In [ ]:
# 시대별 1964 ~ 2022 (59년)
# 2646곡에서 중복제거시 2544
# 300곡씩 파일 저장하기
# songIds가 1964~ 2022까지  set 전 2646
# set(songIds) = 2544개  
# songIds[:300]
# songIds[300:600]
# songIds[600:900]
# songIds[900:1200]
# songIds[1200:1500]
# songIds[1500:1800]
# songIds[1800:2100]
# songIds[2100:2400]
# songIds[2400:]

In [ ]:
infos = []
for songId in tqdm(songIds[300:600]):
    try:
        song_url = f'https://www.melon.com/song/detail.htm?songId={songId}'
        driver.get(song_url)
        time.sleep(2)
        soup = BeautifulSoup(driver.page_source, 'html.parser')
        info = soup.select_one('#conts')
        # songId = info.select_one('#btnLike')['data-song-no']
        title = info.select_one('.song_name').get_text().strip().split('\t')[-1]
        artist = info.select_one('.artist_name').get_text()
        genre = info.select_one('dl > dd:nth-child(6)').get_text().replace(',', '')
        album = info.select_one('.list > dd').get_text()
        ul = info.select('.list_person.clfix > li')
        lyricist = ul[0].select_one('.artist_name').get_text()
        composer = ul[1].select_one('.artist_name').get_text()
        date = info.select_one('dl > dd:nth-child(4)').get_text().replace('.','')
        img = info.select_one('.image_typeAll > img')['src']
        comment = info.select_one('#revCnt').get_text().replace('개', '')
        like = info.select_one('#d_like_count').get_text().replace(',', '')
        lyric = str(info.select_one('.lyric')).split('\t\t\t')[-1].replace('\t', '').replace('<br/>', '\n').split('</div>')[0].strip()
        try:
            buttons = soup.select('.meta > button')
            plylstSeq = ' '.join([button['data-djcol-no'] for button in buttons])
            infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                        'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                        'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})
        except:
            plylstSeq = np.nan
            infos.append({'songId': songId, 'title': title, 'artist': artist, 'genre':genre,
                        'album': album, 'lyricist': lyricist, 'composer': composer, 'date':date, 'img': img,
                        'comment': comment, 'like': like, 'lyric': lyric, 'plylstSeq': plylstSeq})

    except:
        continue

In [6]:
df = pd.DataFrame(infos)
df.tail(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
138,3128693,송학사,김태곤,성인가요/트로트,골든 디럭스 20,김태곤,김태곤,19900101,https://cdnimg.melon.co.kr/cm/album/images/011...,2,398,,454914652 439176495
139,1374303,흐린 가을 하늘에 편지를 써,김광석,포크/블루스,"김광석 다시부르기 1, 2 (Remasterd)",김창기,김창기,20060826,https://cdnimg.melon.co.kr/cm/album/images/003...,3,513,비가 내리면\n음 나를 둘러싸는\n시간의 숨결이 떨쳐질까\n비가 내리면\n음 내가 ...,439176590 480935329


In [7]:
df.shape[0], df.songId.nunique()

(140, 140)

In [8]:
df.to_csv('data/melon_year1.csv', index=False, encoding='utf-8')
pd.read_csv('data/melon_year1.csv').head(2)

,songId,title,artist,genre,album,lyricist,composer,date,img,comment,like,lyric,plylstSeq
0,1936373,영등포의 밤,오기택,성인가요/트로트,"20골드 (영등포의 밤,바닷가에서)",김부해,김부해,19700101,https://cdnimg.melon.co.kr/cm/album/images/003...,0,44,NaN,460454132 518268022
1,57313,미소,서유석,포크/블루스,서유석 골든 힛트 앨범,주석천 (Sok-Chon Joo),주석천 (Sok-Chon Joo),19781025,https://cdnimg.melon.co.kr/cm/album/images/000...,0,37,NaN,460454148 483794266
